In [1]:
from cag.embeddings import SentenceTransformerEmbeddings
from cag.models import ChatOllama

embeddings_model = SentenceTransformerEmbeddings('sentence-transformers/all-mpnet-base-v2')
qwen = ChatOllama(model='qwen2.5', temprature=0.001)
llama = ChatOllama(model='llama3.2', temprature=0.001)

C:\Users\Laptopkaran\AppData\Local\Programs\Python\Python312\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


# Speed Test of CAG on Intel Core i7 1165G7

In [2]:
import json
import datasets

with open('F:\\OneDrive\\Desktop\\Research\\Dataset\\CRSB-Texts.json', 'r') as f:
    crsb = json.load(f)

crsb = crsb['amazon_rainforest']


squad = datasets.load_dataset('rajpurkar/squad')
squad = squad['validation'].shuffle()
squad = squad[:100]

contexts = crsb['contents']
questions = squad['question']

Using the latest cached version of the dataset since rajpurkar/squad couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'plain_text' at F:\Models\huggingface\datasets\rajpurkar___squad\plain_text\0.0.0\7b6d24c440a36b6815f21b70d25016731768db1f (last modified on Thu Nov 28 23:06:36 2024).


In [34]:
from cag.vector_candidates.vc import VectorCandidates
from cag.gate.vector_candidates import VectorCandidatesGate

with open('F:\\OneDrive\\Desktop\\Research\\Dataset\\CRSB-Embeddings-MPNET.json', 'r') as f:
    crsb = json.load(f)

crsb_contexts_embeddings = crsb['amazon_rainforest']['contents']
crsb_pseudo_queries_embeddings = crsb['amazon_rainforest']['questions']
VC = VectorCandidates(contexts=[crsb_contexts_embeddings], questions=[crsb_pseudo_queries_embeddings])

Gate = VectorCandidatesGate(vc=VC, embedding_model=embeddings_model)

In [48]:
from time import time
import numpy as np

results = []
acc = []

for i, question in enumerate(questions):
    a = time()
    _, needs_retrieval = Gate(question, policy= 95, threshold= 0.0)
    b = time()
    acc.append(needs_retrieval)
    results.append(b - a)
    

In [49]:
print(f'Mean of the execution : {np.mean(np.array(results))}')

Mean of the execution : 0.040327818393707277


In [50]:
print(acc.count(False))

100


# Speed Test of Adaptive-RAG on Llama 3.2 on Ollama - Intel Core i7 1165G7

In [51]:
def query_classification(context, query):
    prompt = f"""I will give you a context and a query . please determine if the context is related to the query or not. this means that the context can potentially answer the query or not. 
    first provide the reasons that why the context is related to the query or not; then result a YES or NO answer which determines the context can answer the question or not.
    
    This is the context :
    Context : {context}
    
    Here is the Query :
    Query : {query}
    """
    
    prompt = prompt.format(context = context, query = query)
    
    result = llama.invoke(prompt).content
    
    result = result.lower()
    
    score = True if result.count('yes') > result.count('no') else False
    
    return result, score

In [52]:
import numpy as np
from time import time

adapt_reasons = []
classified = []
times = []

for i, question in enumerate(questions):
    
    a = time()
    x, y = query_classification(contexts[i], question)
    adapt_reasons.append(x)
    classified.append(y)
    b = time()
    
    times.append(b-a)
    
    print(f'Question {i} processed in {b - a} seconds')
    
print(np.mean(np.array(times)))

Question 0 processed in 16.249415159225464 seconds
Question 1 processed in 13.60577654838562 seconds
Question 2 processed in 10.488154649734497 seconds
Question 3 processed in 13.914227962493896 seconds
Question 4 processed in 14.02314281463623 seconds
Question 5 processed in 16.04472279548645 seconds
Question 6 processed in 13.128191471099854 seconds
Question 7 processed in 13.03741979598999 seconds
Question 8 processed in 9.17011547088623 seconds
Question 9 processed in 10.818281412124634 seconds
Question 10 processed in 15.283581972122192 seconds
Question 11 processed in 16.783185958862305 seconds
Question 12 processed in 25.952962398529053 seconds
Question 13 processed in 21.270318508148193 seconds
Question 14 processed in 16.901559114456177 seconds
Question 15 processed in 13.20402717590332 seconds
Question 16 processed in 27.559168100357056 seconds
Question 17 processed in 19.753660202026367 seconds
Question 18 processed in 15.804246425628662 seconds
Question 19 processed in 19.0